Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic regression

In [24]:
batch_size = 128
beta = 0.1

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  t = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = t + beta*tf.nn.l2_loss(t)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [25]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 28.384045
Minibatch accuracy: 9.4%
Validation accuracy: 16.1%
Minibatch loss at step 500: 1.194015
Minibatch accuracy: 80.5%
Validation accuracy: 74.9%
Minibatch loss at step 1000: 1.236103
Minibatch accuracy: 77.3%
Validation accuracy: 76.3%
Minibatch loss at step 1500: 0.703930
Minibatch accuracy: 84.4%
Validation accuracy: 77.0%
Minibatch loss at step 2000: 0.620272
Minibatch accuracy: 85.2%
Validation accuracy: 77.4%
Minibatch loss at step 2500: 0.716988
Minibatch accuracy: 79.7%
Validation accuracy: 77.8%
Minibatch loss at step 3000: 1.102662
Minibatch accuracy: 78.1%
Validation accuracy: 78.5%
Test accuracy: 86.5%


### Neural network

In [39]:
def prediction(tf_dataset, W1, b1, W2, b2):
    Y1 = tf.matmul(tf_dataset, W1) + b1
    X2 = tf.nn.relu(Y1)
    Y2 = tf.matmul(X2, W2) + b2
    return tf.nn.softmax(Y2)

batch_size = 128
num_nodes = 1024
beta = 1e-4

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
  b1 = tf.Variable(tf.zeros([num_nodes]))
    
  W2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  Y1 = tf.matmul(tf_train_dataset, W1) + b1
  X2 = tf.nn.relu(Y1)
  logits = tf.matmul(X2, W2) + b2
  t = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = t + beta*tf.nn.l2_loss(t)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = prediction(tf_valid_dataset, W1, b1, W2, b2)
  test_prediction = prediction(tf_test_dataset, W1, b1, W2, b2)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 323.479950
Minibatch accuracy: 10.2%
Validation accuracy: 27.9%
Minibatch loss at step 500: 28.477230
Minibatch accuracy: 77.3%
Validation accuracy: 78.7%
Minibatch loss at step 1000: 9.264588
Minibatch accuracy: 81.2%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 8.350443
Minibatch accuracy: 85.9%
Validation accuracy: 79.8%
Minibatch loss at step 2000: 4.190209
Minibatch accuracy: 84.4%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 2.832783
Minibatch accuracy: 84.4%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 2.295864
Minibatch accuracy: 83.6%
Validation accuracy: 82.6%
Test accuracy: 89.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [54]:
batch_size = 16
num_nodes = 1024
beta = 1e-4

num_train = 100
mini_train_dataset = train_dataset[:num_train]
mini_train_labels = train_labels[:num_train]

num_steps = 1000

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (mini_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = mini_train_dataset[offset:(offset + batch_size), :]
    batch_labels = mini_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 256.720062
Minibatch accuracy: 18.8%
Validation accuracy: 24.6%
Minibatch loss at step 100: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 59.6%
Minibatch loss at step 200: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 59.6%
Minibatch loss at step 300: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 59.6%
Minibatch loss at step 400: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 59.6%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 59.6%
Minibatch loss at step 600: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 59.6%
Minibatch loss at step 700: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 59.6%
Minibatch loss at step 800: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 59.6%
Minibatch loss at step 900: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 59.6%
Test accuracy: 65.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [61]:
def prediction(tf_dataset, W1, b1, W2, b2):
    Y1 = tf.matmul(tf_dataset, W1) + b1
    X2 = tf.nn.relu(Y1)
    Y2 = tf.matmul(X2, W2) + b2
    return tf.nn.softmax(Y2)

batch_size = 128
num_nodes = 1024
beta = 1e-4

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
  b1 = tf.Variable(tf.zeros([num_nodes]))
    
  W2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
    
  keep_prob = tf.placeholder(tf.float32)
    
  # Training computation.
  Y1 = tf.matmul(tf_train_dataset, W1) + b1
  X2 = tf.nn.relu(Y1)
  X2_drop = tf.nn.dropout(X2, keep_prob)

  logits = tf.matmul(X2_drop, W2) + b2
  t = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = t + beta*tf.nn.l2_loss(t)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = prediction(tf_valid_dataset, W1, b1, W2, b2)
  test_prediction = prediction(tf_test_dataset, W1, b1, W2, b2)

num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 433.331818
Minibatch accuracy: 11.7%
Validation accuracy: 21.1%
Minibatch loss at step 500: 44.730465
Minibatch accuracy: 75.0%
Validation accuracy: 80.4%
Minibatch loss at step 1000: 19.658358
Minibatch accuracy: 72.7%
Validation accuracy: 79.9%
Minibatch loss at step 1500: 13.793810
Minibatch accuracy: 73.4%
Validation accuracy: 79.9%
Minibatch loss at step 2000: 6.025596
Minibatch accuracy: 77.3%
Validation accuracy: 80.3%
Minibatch loss at step 2500: 5.260529
Minibatch accuracy: 75.0%
Validation accuracy: 78.9%
Minibatch loss at step 3000: 2.793600
Minibatch accuracy: 75.0%
Validation accuracy: 79.8%
Minibatch loss at step 3500: 10.549148
Minibatch accuracy: 71.9%
Validation accuracy: 78.2%
Minibatch loss at step 4000: 6.338033
Minibatch accuracy: 79.7%
Validation accuracy: 78.8%
Minibatch loss at step 4500: 2.404853
Minibatch accuracy: 71.1%
Validation accuracy: 80.6%
Minibatch loss at step 5000: 5.275173
Minibatch accuracy: 78.9%
Validation a

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
